In [ ]:
%run '../lib/imports.ipynb'

In [ ]:
modelTypeName = "ResNet_valueMinMaxScaled"

In [ ]:
dataColumnName = 'valueMinMaxScaled'

In [ ]:
%run '../lib/utils_anomaly_detection.ipynb'

In [ ]:
# load datasets
data = pd.read_csv('../../data/rq_1-3_train_test/2024-01-15_10-51-45__2024-01-15_14-26-45_load-gen-msg-w-spikes-10s-rate.csv', 
                   index_col=['EventDateTime'], parse_dates=['EventDateTime'])
dataLatency = pd.read_csv('../../data/rq_1-3_train_test/2024-01-15_10-51-45__2024-01-15_14-26-45_load-gen-avg-latency-10s-rate.csv', 
                          index_col='EventDateTime', parse_dates=['EventDateTime'])

In [ ]:
%run '../lib/prepareDataSet.ipynb'

In [ ]:
%run '../lib/model_commons.ipynb'

In [ ]:
start_time = datetime.datetime.now()

In [ ]:
def create_model(config):
    
    model = Sequential()

    model.add(Input(shape=(config["T"],1)))
    for i in range(config["n_layers"]):
        #add the ResNet layers
        model.add(Conv1D(filters=config["n_neurons"], kernel_size=config["conv_kernel"], padding="same"))
        model.add(BatchNormalization())
        model.add(ReLU())
        
    model.add(GlobalAveragePooling1D())

    for i in range(config["n_dense_layers"]-1):
        model.add(Dense(config["n_neurons"], activation="relu"))
        model.add(Dropout(rate=config["dense_dp"]))

    model.add(Dense(1))
            
    return model

In [ ]:
####
# Set the target environment to cluster or local. Default is local
####
run_local = True

In [ ]:
if run_local == False:
    print("Running on Cluster")
    from codeflare_sdk import TokenAuthentication
    from codeflare_sdk import generate_cert
    
    auth = TokenAuthentication(
        token = "<your_access_token>", # execute ocp whoami -t on the authenticated cluster to obtain the token
        server = "<OCP cluster API URL>",
        skip_tls = True
    )
    auth.login()
    
    # Create required TLS cert and export the environment variables to enable TLS
    generate_cert.generate_tls_cert('raycluster-complete', 'raycluster')
    generate_cert.export_env('raycluster-complete', 'raycluster')
    
    ray_endpoint = 'ray://raycluster-complete-head-svc.raycluster.svc.cluster.local:10001' # ensure your ray cluster URL is correct
    ray.shutdown()
    ray.init(address=ray_endpoint, logging_level=logging.ERROR, log_to_driver=False)
else:
    ray.shutdown()
    ray.init()    

In [ ]:
results_resnet = tune_model(1, 10)

In [ ]:
result = results_resnet.get_best_result()

In [ ]:
results_n_step_predict = {}
# for result in results_resnet:
try:
    model_name = result.metrics['model_name']
    T = result.metrics['T']
    run_id = result.metrics['run_id']
    for predict_ahead in [5, 10, 15, 30, 45, 60, 90, 120]: 
        res = run_n_step_evaluation.remote(model_name, run_id, T, predict_ahead)
        tag = f'runID_{run_id}-model_{model_name}-T_{T}-p_ahead_{predict_ahead}'
        results_n_step_predict[tag] = res
        print(f'Scheduled job for T:{T}, predict_ahead: {predict_ahead}, model_name:{model_name}, run_id:{run_id}')    
except:
    print(f'ERROR scheduling job for T:{T}, predict_ahead: {predict_ahead}, model_name:{model_name}, run_id:{run_id}')

In [ ]:
ex_n_step = {}
for item in results_n_step_predict.keys():
    try:
        res = ray.get(results_n_step_predict[item])
    except:
        print(f'Error getting results for key:{item}')
        res = None
        
    ex_n_step[item]=res

In [ ]:
len(ex_n_step.keys())

In [ ]:
ex_n_step

In [ ]:
ray.shutdown()

In [ ]:
end_time = datetime.datetime.now()

In [ ]:
time_difference = (end_time - start_time).total_seconds()

In [ ]:
print(f"Experiment runtime was {time_difference} seconds")